In [3]:
#setup imports and make ure the files we care about exist
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os.path
import feature_extraction as fe

fe.test_cases()

Nothing here yet


/home/jaywalker/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3267: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
#first, load the dataset

#pick out the attack examples

#convert attack examples into time windows for training

#convert each window into appropriate feature tensor

#associate a "masking" vector with each window

#train GAN to replace masked packets

#observe results